# 1. Import Libraries and Load Files

In [17]:
# Import libraries
import tarfile
import pandas as pd
import json
import pyarrow as pa
import pyarrow.parquet as pq
import os

In [ ]:
# Set path
tar_path = r"C:\Users\user\Desktop\Yelp_data_analysis\datasets\yelp_dataset.tar"
extract_path = r"C:\Users\user\Desktop\Yelp_data_analysis\datasets\json_file"

# Unzip files
with tarfile.open(tar_path, "r") as tar:
    tar.extractall(path=extract_path)

# 2. Convert JSON File to Parquet

In [19]:
# -------------------------
# 1. Convert business file json to parquet
# -------------------------

# Load json file
business = pd.read_json("../datasets/json_file/yelp_academic_dataset_business.json", lines=True)

# --- A) Filter only rows containing 'Restaurant' ---
business = business[business['categories'].str.contains('Restaurant', case=False, na=False)]

# --- B) Filter out unnecessary states ---
exclude_states = ['AB', 'NC', 'CO', 'HI', 'MT', 'XMS']
business = business[~business['state'].isin(exclude_states)]

In [20]:
# --- C) Filter out non-restaurant categories ---

# Check unique categories
categories_series = business['categories'].dropna()

unique_categories = set(
    cat.strip()
    for cats in categories_series
    for cat in cats.split(',')
)

unique_categories = sorted(unique_categories)

print(unique_categories) 

['Acai Bowls', 'Accessories', 'Accountants', 'Active Life', 'Acupuncture', 'Adult', 'Adult Education', 'Adult Entertainment', 'Advertising', 'Afghan', 'African', 'Air Duct Cleaning', 'Airlines', 'Airport Lounges', 'Airport Shuttles', 'Airport Terminals', 'Airports', 'Amateur Sports Teams', 'American (New)', 'American (Traditional)', 'Amusement Parks', 'Animal Shelters', 'Antiques', 'Apartments', 'Appliances', 'Appliances & Repair', 'Aquariums', 'Arabic', 'Arcades', 'Argentine', 'Armenian', 'Art Classes', 'Art Galleries', 'Art Museums', 'Art Tours', 'Arts & Crafts', 'Arts & Entertainment', 'Asian Fusion', 'Auction Houses', 'Audio/Visual Equipment Rental', 'Australian', 'Austrian', 'Auto Customization', 'Auto Detailing', 'Auto Glass Services', 'Auto Loan Providers', 'Auto Parts & Supplies', 'Auto Repair', 'Automotive', 'Axe Throwing', 'Bagels', 'Bakeries', 'Bangladeshi', 'Banks & Credit Unions', 'Bar Crawl', 'Barbeque', 'Barbers', 'Bars', 'Bartenders', 'Basque', 'Battery Stores', 'Battin

In [21]:
# Define non-food-related categories (to be excluded)
non_food_categories = [
    # Business / Professional services
    'Accountants', 'Advertising', 'Business Consulting', 'Lawyers', 'Legal Services', 
    'Financial Services', 'Real Estate', 'Contractors', 'Marketing', 'IT Services & Computer Repair',

    # Beauty / Health
    'Hair Salons', 'Nail Salons', 'Massage', 'Spas', 'Chiropractors', 'Medical Centers',
    'Doctors', 'Dentists', 'Hospitals', 'Plastic Surgeons', 'Cosmetic Surgeons', 'Opticians',
    'Nutritionists', 'Weight Loss Centers', 'Yoga', 'Gyms', 'Fitness & Instruction',

    # Retail / Shopping
    'Books', 'Bookstores', 'Clothing', "Men's Clothing", "Women's Clothing", 
    'Jewelry', 'Gift Shops', 'Toy Stores', 'Electronics', 'Furniture Stores',
    'Hardware Stores', 'Sporting Goods', 'Bike Shops', 'Car Dealers',

    # Entertainment / Leisure
    'Music Venues', 'Dance Clubs', 'Casinos', 'Museums', 'Art Galleries', 'Comedy Clubs', 'Zoos',
    'Amusement Parks', 'Bowling', 'Golf', 'Movie Theaters', 'Theaters', 'Escape Games', 'Billiards',

    # Accommodation / Travel / Real Estate
    'Hotels', 'Motels', 'Resorts', 'Bed & Breakfast', 'Vacation Rentals', 'Travel Services',

    # Automotive
    'Auto Repair', 'Car Wash', 'Auto Parts & Supplies', 'Motorcycle Dealers', 
    'Oil Change Stations', 'Gas Stations', 'Car Dealerships',

    # Education / Government / Religion
    'Colleges & Universities', 'Elementary Schools', 'Adult Education', 'Special Education',
    'Churches', 'Religious Organizations', 'Public Services & Government',

    # Home & Miscellaneous Services
    'Home Cleaning', 'Laundry Services', 'Dry Cleaning', 'Plumbing', 'Electricians',
    'Landscaping', 'Security Systems', 'Roofing', 'Moving Services', 'Storage', 'Shipping Centers',
]

# Keep only restaurant-related rows
business = business[
    ~business['categories'].fillna('').apply(lambda x: any(cat in x for cat in non_food_categories))
]


In [22]:
# Save as parquet file
business.to_parquet("../datasets/parquet_file/business.parquet", engine="pyarrow")

In [23]:
# -------------------------
# 2. Convert checkin file json to parquet
# -------------------------

# Load json file
checkin = pd.read_json("../datasets/json_file/yelp_academic_dataset_checkin.json", lines=True)

# Filter only rows that match the business_id
checkin = checkin[checkin["business_id"].isin(business["business_id"])]

# Save as parquet file
checkin.to_parquet("../datasets/parquet_file/check_in.parquet", engine="pyarrow")

In [24]:
# -------------------------
# 3. Convert review file json to parquet
# -------------------------

# Read a large JSONL file line by line
def convert_jsonl_skip_bad_lines(json_path, parquet_path, chunk_size=100_000):

    os.makedirs(os.path.dirname(parquet_path), exist_ok=True)

    good_tables = []        # arrow tables for good data
    buffer_lines = []       # raw lines before parsing
    total_kept = 0
    total_bad = 0
    chunk_id = 0

    def flush_lines_to_table(lines_buffer, chunk_id):
        nonlocal total_kept, total_bad

        records = []
        for raw in lines_buffer:
            try:
                rec = json.loads(raw)
                records.append(rec)
            except json.JSONDecodeError:
                total_bad += 1
                continue  # skip single bad line

        if not records:
            return None

        df_chunk = pd.DataFrame(records)
        total_kept += len(df_chunk)

        table = pa.Table.from_pandas(df_chunk, preserve_index=False)
        print(f"✅ Chunk {chunk_id}: kept {len(df_chunk):,} rows (bad lines so far: {total_bad:,})")
        return table

    with open(json_path, "r", encoding="utf-8") as f:
        for line_idx, raw_line in enumerate(f, start=1):
            buffer_lines.append(raw_line)

            if line_idx % chunk_size == 0:
                chunk_id += 1
                table = flush_lines_to_table(buffer_lines, chunk_id)
                if table is not None:
                    good_tables.append(table)
                buffer_lines = []

        # flush last partial chunk
        if buffer_lines:
            chunk_id += 1
            table = flush_lines_to_table(buffer_lines, chunk_id)
            if table is not None:
                good_tables.append(table)

    if good_tables:
        full_table = pa.concat_tables(good_tables, promote=True)
        pq.write_table(full_table, parquet_path)
        print("🎉 Done!")
        print(f"   Total valid rows saved : {full_table.num_rows:,}")
        print(f"   Total broken lines dropped: {total_bad:,}")
        print(f"   Saved to {parquet_path}")
    else:
        print("❌ No valid data ❌")

# Save review_raw file
convert_jsonl_skip_bad_lines(
    "../datasets/json_file/yelp_academic_dataset_review.json",
    "../datasets/parquet_file/review_raw.parquet",
    chunk_size=100_000
)


✅ Chunk 1: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 2: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 3: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 4: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 5: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 6: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 7: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 8: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 9: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 10: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 11: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 12: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 13: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 14: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 15: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 16: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 17: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 18: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 19: kept 100,000 rows (bad lines so far: 0)
✅ Chunk 20: kept 100,

C:\Users\user\AppData\Local\Temp\ipykernel_10448\961885345.py:67: FutureWarning: promote has been superseded by promote_options='default'.
  convert_jsonl_skip_bad_lines(


🎉 Done!
   Total valid rows saved : 4,553,912
   Total broken lines dropped: 1
   Saved to ../datasets/parquet_file/review_raw.parquet


In [25]:
# Load Raw Review Parquet file
review = pd.read_parquet("../datasets/parquet_file/review_raw.parquet")

# Filter only rows that match the restaurant business_id
review = review[review["business_id"].isin(business["business_id"])]

# Save the filtered dataset as a new Parquet file
review.to_parquet("../datasets/parquet_file/review.parquet", engine="pyarrow")

In [26]:
# -------------------------
# 4. Convert tip file json to parquet
# -------------------------

# Load json file
tip = pd.read_json("../datasets/json_file/yelp_academic_dataset_tip.json", lines=True)

# Filter only rows that match the business_id
tip = tip[tip["business_id"].isin(business["business_id"])]

# Save as parquet file
tip.to_parquet("../datasets/parquet_file/tip.parquet", engine="pyarrow")

In [27]:
# -------------------------
# 5. Convert user file json to parquet
# -------------------------

# Load the filtered review parquet file
review = pd.read_parquet("../datasets/parquet_file/review.parquet")

# Extract unique user IDs from review table
review_user_ids = set(review["user_id"].unique())

# Read user.json in chunks
user_chunks = pd.read_json("../datasets/json_file/yelp_academic_dataset_user.json", lines=True, chunksize=100000)

# Filter each chunk by user_id and store results
filtered_list = []

for i, chunk in enumerate(user_chunks):
    filtered_chunk = chunk[chunk["user_id"].isin(review_user_ids)]
    filtered_list.append(filtered_chunk)
    print(f"✅ Chunk {i+1} processed ({len(filtered_chunk)} rows kept)")

# Concatenate all filtered chunks into a single DataFrame
filtered_user = pd.concat(filtered_list, ignore_index=True)

# Save the filtered user table as Parquet
filtered_user.to_parquet("../datasets/parquet_file/user.parquet", engine="pyarrow")


✅ Chunk 1 processed (88231 rows kept)
✅ Chunk 2 processed (82696 rows kept)
✅ Chunk 3 processed (79164 rows kept)
✅ Chunk 4 processed (69757 rows kept)
✅ Chunk 5 processed (78984 rows kept)
✅ Chunk 6 processed (71961 rows kept)
✅ Chunk 7 processed (73525 rows kept)
✅ Chunk 8 processed (65769 rows kept)
✅ Chunk 9 processed (69490 rows kept)
✅ Chunk 10 processed (62889 rows kept)
✅ Chunk 11 processed (65546 rows kept)
✅ Chunk 12 processed (59005 rows kept)
✅ Chunk 13 processed (61299 rows kept)
✅ Chunk 14 processed (54881 rows kept)
✅ Chunk 15 processed (42136 rows kept)
✅ Chunk 16 processed (32790 rows kept)
✅ Chunk 17 processed (0 rows kept)
✅ Chunk 18 processed (0 rows kept)
✅ Chunk 19 processed (0 rows kept)
✅ Chunk 20 processed (0 rows kept)
